In [5]:
# https://github.com/DangerousFreedom1984/monero_inflation_checker/blob/main/dumber25519.py
# https://doc.libsodium.org/advanced/point-arithmetic#elligator-2-map
# https://archive.is/yfINb

import nacl.bindings

from Cryptodome.Hash import keccak
from binascii import hexlify, unhexlify


q = 2**255 - 19
d = -121665 * pow(121666, -1, q) % q


def keccak_256(data) -> str:
    return keccak.new(digest_bits=256).update(data).digest()


def inv(x):
    return pow(x, -1, q)


def modp_inv(x):
    return pow(x, q - 2, q)


def sqroot(xx):
  x = pow(xx, (q+3)//8, q)
  if (x*x - xx) % q != 0:
    I = pow(2, (q-1)//4, q) 
    x = (x*I) % q
  if (x*x - xx) % q != 0: 
    print("no square root!")
  return x


def compress_point(x, y):
    parity = x & 1
    compressed_y = y.to_bytes(32, byteorder='little')
    if parity:
        msb = compressed_y[-1] | 0x80
        compressed_y = compressed_y[:-1] + bytes([msb])
    return hexlify(compressed_y).decode()


def mul_point_8(p):
    point_add = nacl.bindings.crypto_core_ed25519_add
    p2 = p
    for i in range(7):
        p2 = point_add(p, p2)
    return hexlify(p2).decode()


def hash_to_point(hexVal: bytes) -> str:
    u = int.from_bytes(keccak_256(hexVal), byteorder="little") % q
    A = 486662
    sqrtm1 = sqroot(-1)
    w = (2 * u * u + 1) % q
    xp = (w *  w - 2 * A * A * u * u) % q
    rx = pow(w * inv(xp), (q+3)//8, q)

    x = rx * rx * (w * w - 2 * A * A * u * u) % q
    y = (2 * u * u  + 1 - x) % q

    negative = False
    if (y != 0):
        y = (w + x) % q
        if (y != 0) :
            negative = True
        else :
            rx = rx * -1 * sqroot(-2 * A * (A + 2) ) % q
            negative = False
    else :
        rx = (rx * -1 * sqroot(2 * A * (A + 2) ) ) % q 
    if not negative:
        rx = (rx * u) % q
        z = (-2 * A * u * u)  % q
        sign = 0
    else:
        z = -1 * A
        x = x * sqrtm1 % q
        y = (w - x) % q 
        if (y != 0) :
            rx = rx * sqroot( -1 * sqrtm1 * A * (A + 2)) % q
        else :
            rx = rx * -1 * sqroot( sqrtm1 * A * (A + 2)) % q
        sign = 1
    #setsign
    if ( (rx % 2) != sign ):
        rx =  - (rx) % q 
    rz = (z + w) % q
    ry = (z - w) % q
    rx = rx * rz % q

    rz_inv = modp_inv(rz)
    x = rx * rz_inv % q
    y = ry * rz_inv % q
    p = compress_point(x,y)
    return mul_point_8(unhexlify(p))

In [6]:
import nacl.bindings

scalarmult = nacl.bindings.crypto_scalarmult_ed25519_noclamp


stealth_address_private_key = "9c5a754f43e4e65ee525ca56f813dd9ddf75cd59e7b24c2aae6ed6dcc661cb06"
stealth_address_public_key = "6f598be2c3c473ccff7ad1fb42ed46660bae0ea353d71192afe1a520aacb9374"
hashed_stealth_address_public_key = hash_to_point(unhexlify(stealth_address_public_key)) # "a05f0062aea80f577d029bb93683d3c55fa8378510e9a7350cc2a778814ee1dd"
hexlify(scalarmult(unhexlify(stealth_address_private_key), unhexlify(hashed_stealth_address_public_key))).decode() # "5631d2eacb1d2c88ba2e4625604c0312d33e156727448db8e2f55b5e4f83bd01"

'5631d2eacb1d2c88ba2e4625604c0312d33e156727448db8e2f55b5e4f83bd01'

In [7]:
# https://loup-vaillant.fr/tutorials/cofactor
# https://github.com/LoupVaillant/Monocypher/blob/master/tests/gen/elligator.py


import nacl.bindings

from Cryptodome.Hash import keccak
from binascii import hexlify, unhexlify


def create_key_image(stealth_address_public_key: str, stealth_address_private_key: str) -> str:
    
    # convert hex strings to bytes
    stealth_address_public_key_bytes = unhexlify(stealth_address_public_key)
    stealth_address_private_key_bytes = unhexlify(stealth_address_private_key)

    # hash public key, convert to integer modulus prime field, convert back to bytes
    hashed_stealth_address_public_key = keccak.new(digest_bits=256).update(stealth_address_public_key_bytes).digest()
    q = 2**255 - 19
    hash_as_int = int.from_bytes(hashed_stealth_address_public_key, byteorder="little") % q
    hash_as_int = hash_as_int.to_bytes(32, byteorder="little")

    # use nacl bindings to map modified hashed value to a curve point
    curve_point_from_hash = nacl.bindings.crypto_core_ed25519_from_uniform(hash_as_int)
    
    # need to review: nacl compression is resulting in a different encoding than Monero
    msb = curve_point_from_hash[-1] | 0x80
    modified_curve_point_from_hash = curve_point_from_hash[:-1] + bytes([msb])

    # calculate key image
    key_image = nacl.bindings.crypto_scalarmult_ed25519_noclamp(stealth_address_private_key_bytes, modified_curve_point_from_hash)
    return hexlify(key_image).decode()


create_key_image(stealth_address_public_key, stealth_address_private_key)

'5631d2eacb1d2c88ba2e4625604c0312d33e156727448db8e2f55b5e4f83bd01'